In [1]:
#libraries 
import os
import numpy as np
import torch
import glob
import torch.nn as nn
from torchvision.transforms import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.autograd import Variable
import torchvision
import pathlib

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
             

In [3]:
print(device)

cpu


In [4]:
#data processing
transformer = transforms.Compose([
    transforms.Resize((150,150)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),  #augmentation techniques to artifically increase the sample size\
    transforms.ToTensor(), #PIL image to tensor image with normal pixel range of 0-225 to 0-1
    transforms.Normalize([0.5,0.5,0.5],
                         [0.5,0.5,0.5]) #changes the range from 0-1 to -1 to 1   
                                                      #formula: (x-mean)/standard deviation
    
])


In [5]:
#loading data (called dataloaders)

#paths for training and testing
train_path = 'C:\\Users\\hqi\\Documents\\seperated data\\training data'
test_path = 'C:\\Users\\hqi\\Documents\\seperated data\\test data'
#dataloader for training data set
train_loader = DataLoader(
    torchvision.datasets.ImageFolder(train_path, transform = transformer),
    batch_size = 64, shuffle = True #shuffle to make sure the model is not biased towards certain categories
    #batch size used to be 256
)

#dataloader for testing data set
test_loader = DataLoader(
    torchvision.datasets.ImageFolder(test_path, transform = transformer),
    batch_size = 32, shuffle = True #shuffle to make sure the model is not biased towards certain categories
)

In [6]:
#fetch all of the classes 
root = pathlib.Path(train_path)
classes = sorted([j.name.split('/')[-1] for j in root.iterdir()])

In [7]:
print(classes)

['benign', 'malignant - melanoma']


In [8]:
#Convolutional neural network class

class ConvNet(nn.Module):
    
    
    #specify all of the layers in the network in the constructor
    def __init__(self, num_classes = 6):
        super(ConvNet,self).__init__()
        
        #input shape: (128, 3, 300, 300)
        
        #apply filter
        self.conv1=nn.Conv2d(in_channels=3,out_channels=12,kernel_size=3,stride=1,padding=1)
        #output size after convolutional filter is applied is represented by this equation:
        # ((w-f+2P)/s) +1
        #Current Shape: (128, 12, 300, 300)
        self.bn1 = nn.BatchNorm2d(num_features = 12)
        self.relu1=nn.LeakyReLU()
        #Current Shape: (128, 12, 300, 300)
        
        self.conv2=nn.Conv2d(in_channels=12,out_channels=20,kernel_size=3,stride=1,padding=1)
        #Current Shape: (128, 20, 150, 150)
        self.relu2=nn.ReLU()
        #Current Shape: (128, 20, 150, 150)

        
        


        
        
        self.fc=nn.Linear(in_features=150 * 150 * 20,out_features=num_classes)
       
    
    
    #Feedforward function:
    def forward(self,input):
        output=self.conv1(input)
        output=self.bn1(output)
        output=self.relu1(output)
            
        output=self.conv2(output)
        output=self.relu2(output)

        
        
        

        
        #shape: (256, 52, 75, 75)
        #change shape of matrix to feed into the fc layer
        
        output = output.view(-1, 20*150*150)
        output =self.fc(output)
        return output
        
        
        

In [9]:
model = ConvNet(num_classes = 6).to(device)

In [10]:
#Optimizer
optimizer=Adam(model.parameters(),lr=0.01,weight_decay=0.0001)
loss_function=nn.CrossEntropyLoss()


In [11]:
num_epochs = 50

In [12]:
#Calc size of training and testing images
train_count=len(glob.glob(train_path+'/**/*.jpg'))
test_count=len(glob.glob(test_path+'/**/*.jpg'))

In [13]:
print(train_count,test_count)

900 379


In [14]:
#Training model and saving the most accurate model
#Save the model for the epoch that gives the best testing accuracy

best_accuracy = 0

for epoch in range(num_epochs):
    
    #Training on training data set
    model.train()
    train_accuracy = 0.0
    train_loss = 0.0
    for i, (images,labels) in enumerate(train_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
        
        optimizer.zero_grad()
        outputs = model(images)
        
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        
        #increment for train_loss
        train_loss+= loss.cpu().data*images.size(0)
        _,prediction=torch.max(outputs.data,1)
        
        #increment for train_accuracy
        train_accuracy+=int(torch.sum(prediction==labels.data))
    
    train_accuracy = train_accuracy/train_count
    train_loss = train_loss/train_count
        
    
    #Evaluating using testing data set
    model.eval()
    test_accuracy = 0.0
    for i, (images,labels) in enumerate(test_loader):
        if torch.cuda.is_available():
            images=Variable(images.cuda())
            labels=Variable(labels.cuda())
        
        outputs = model(images)
        _,prediction=torch.max(outputs.data,1)
        
        #increment for train_accuracy
        test_accuracy+=int(torch.sum(prediction==labels.data))
    
    test_accuracy = test_accuracy/test_count
    
    print('Epoch: '+str(epoch)+' Train Loss: '+str(train_loss)+' Train Accuracy: '+str(train_accuracy)+' Test Accuracy: '+str(test_accuracy))
    
    
    
    
    #save best model
    
    if test_accuracy > best_accuracy:
        #save the best model
        torch.save(model.state_dict(),'best_few_filters_checkpoint.model')
        #update the value of best_accuracy
        best_accuracy = test_accuracy
    
    

Epoch: 0 Train Loss: tensor(45.9762) Train Accuracy: 0.64 Test Accuracy: 0.8073878627968337
Epoch: 1 Train Loss: tensor(1.3139) Train Accuracy: 0.7833333333333333 Test Accuracy: 0.8021108179419525
Epoch: 2 Train Loss: tensor(0.8406) Train Accuracy: 0.7588888888888888 Test Accuracy: 0.7994722955145118
Epoch: 3 Train Loss: tensor(0.7748) Train Accuracy: 0.7788888888888889 Test Accuracy: 0.7783641160949868
Epoch: 4 Train Loss: tensor(0.7197) Train Accuracy: 0.8088888888888889 Test Accuracy: 0.7862796833773087
Epoch: 5 Train Loss: tensor(0.6849) Train Accuracy: 0.7966666666666666 Test Accuracy: 0.8047493403693932
Epoch: 6 Train Loss: tensor(0.6034) Train Accuracy: 0.8055555555555556 Test Accuracy: 0.7968337730870713
Epoch: 7 Train Loss: tensor(0.6229) Train Accuracy: 0.8122222222222222 Test Accuracy: 0.7941952506596306
Epoch: 8 Train Loss: tensor(0.5537) Train Accuracy: 0.8022222222222222 Test Accuracy: 0.7968337730870713
Epoch: 9 Train Loss: tensor(0.4895) Train Accuracy: 0.82444444444444